<a href="https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/Voice_Cloning_for_Chinese_Speech_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 全新中文声音克隆 Voice Cloning for Chinese Speech

## 配置环境 Set up

In [1]:
! nvidia-smi # 需要使用GPU运行

Sun Aug 27 14:10:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install openai-whisper
! pip install modelscope
! pip install tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install typeguard==2.3.1
! pip install sox
! pip install bitstring
! pip install pysptk --no-build-isolation
! pip install kantts -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install pytorch_wavelets
! pip install tensorboardX
! git clone https://github.com/fbcotter/pytorch_wavelets
! pip install matplotlib
! pip install numpy==1.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796907 sha256=28fb6d17d817dae9ec5c67db1738b946523e588d53e9ef65baf02f9142a80139
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.3/274.3 kB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.13.3
    Uninstalling typeguard-2.13.3:
      Successfully uninstalled typeguard-2.13.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysptk: filename=pysptk-0.2.1-cp310-cp310-linux_x86_64.whl size=1137272 sha256=ef14ac137fd0a5e3d360e6d3d54578aebe2e7f830d70796cf0f874d6627ffa5a
  Stored in directory: /root/.cache/pip/wheels/aa/96/e5/2348f6f568b9888d3505a97dc53287152ffd0440526cc6105d
Successfully built pysptk
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 kB 238.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.8 MB/s eta 0:00:00
 

In [3]:
!apt-get install sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 16 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,760 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3 amd64 14.4.2+git20190427-2+deb11u2build0.22.04.1 [240 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 l

In [4]:
cd pytorch_wavelets

/content/pytorch_wavelets


In [5]:
pip install .

Processing /content/pytorch_wavelets
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=54852 sha256=4a34d2440df4f650aa963dfc62127481a126243d80d3032b9999013d767b8f7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-nxof703y/wheels/f6/77/89/ac0462961f441a5dd4d226216613ef64f3eb2c7f2883efcebf
Successfully built pytorch-wavelets
  Attempting uninstall: pytorch-wavelets
    Found existing installation: pytorch-wavelets 1.3.0
    Uninstalling pytorch-wavelets-1.3.0:
      Successfully uninstalled pytorch-wavelets-1.3.0


In [ ]:
import os
os._exit(00) # 重启notebook

In [1]:
import sox

## 对音频切片处理

In [2]:
import subprocess
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
import torch
import csv
import whisper

def split_long_audio(model, filepaths, character_name, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir) / character_name
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{character_name}_{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))

In [3]:
whisper_size = "medium"
whisper_model = whisper.load_model(whisper_size)

100%|██████████████████████████████████████| 1.42G/1.42G [00:09<00:00, 153MiB/s]


### 请在下方{filename}处填写您上传的wav文件名

In [4]:
split_long_audio(whisper_model, "filename.wav", "test", "dataset_raw") # 请在{filename}处填写您上传的wav文件名

Transcribing file 0: 'nana_speech.wav' to segments...


## 需要新建三个文件夹，分别是："test_wavs", "output_training_data", "pretrain_work_dir"

In [5]:
! mv  -v ./dataset_raw/test/* ./test_wavs/

renamed './dataset_raw/test/test_0_0.wav' -> './test_wavs/test_0_0.wav'
renamed './dataset_raw/test/test_0_10.wav' -> './test_wavs/test_0_10.wav'
renamed './dataset_raw/test/test_0_11.wav' -> './test_wavs/test_0_11.wav'
renamed './dataset_raw/test/test_0_12.wav' -> './test_wavs/test_0_12.wav'
renamed './dataset_raw/test/test_0_13.wav' -> './test_wavs/test_0_13.wav'
renamed './dataset_raw/test/test_0_14.wav' -> './test_wavs/test_0_14.wav'
renamed './dataset_raw/test/test_0_15.wav' -> './test_wavs/test_0_15.wav'
renamed './dataset_raw/test/test_0_16.wav' -> './test_wavs/test_0_16.wav'
renamed './dataset_raw/test/test_0_17.wav' -> './test_wavs/test_0_17.wav'
renamed './dataset_raw/test/test_0_18.wav' -> './test_wavs/test_0_18.wav'
renamed './dataset_raw/test/test_0_19.wav' -> './test_wavs/test_0_19.wav'
renamed './dataset_raw/test/test_0_1.wav' -> './test_wavs/test_0_1.wav'
renamed './dataset_raw/test/test_0_2.wav' -> './test_wavs/test_0_2.wav'
renamed './dataset_raw/test/test_0_3.wav' ->

In [6]:
from modelscope.tools import run_auto_label

2023-08-27 14:16:32,204 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-08-27 14:16:32,208 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-08-27 14:16:32,209 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-08-27 14:16:32,211 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-08-27 14:16:32,271 - modelscope - INFO - Loading done! Current index file version is 1.8.4, with md5 eee6c9359d592cf7de4cce1ebbbe4e16 and a total number of 902 components indexed


In [7]:
input_wav = "./test_wavs/"
output_data = "./output_training_data/"

ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

2023-08-27 14:16:34,729 - modelscope - INFO - Use user-specified model revision: v1.0.7
Downloading: 100%|██████████| 185/185 [00:00<00:00, 764kB/s]
Downloading: 100%|██████████| 706M/706M [00:37<00:00, 19.9MB/s]
Downloading: 100%|██████████| 6.24k/6.24k [00:00<00:00, 2.43MB/s]


2023-08-27 14:17:24
TTS-AutoLabel version: 1.1.8
TTS-AutoLabel resource path: /root/.cache/modelscope/hub/damo/speech_ptts_autolabel_16k/model
Target sampling rate: 16000
Input wav dir: /content/test_wavs
Output data dir: /content/output_training_data
wav_preprocess start...
---  new folder...  ---
---  OK  ---


100%|██████████| 20/20 [00:00<00:00, 90.65it/s]

[VAD] chunk recordings for training.
wav cut by vad start...



  0%|          | 0/20 [00:00<?, ?it/s]

...

100%|██████████| 38/38 [00:00<00:00, 5265.22it/s]

Write prosody file
0 "mismatch" sentences

Trim sp started



100%|██████████| 19/19 [00:00<00:00, 156.63it/s]

Trim sp finished
Auto labeling info: stage 1 | develop mode 0 | gender:female | score 10.000000 | retcode 0
labeling report:
stage 1 | develop mode 0 | gender female | score 10.000000 | retcode 0
qulification report:
credit score: 10.000000
qualified score: 3.000000
normalized snr: 35.000000


abandon utt snr threshold: 10.000000
snr score ration: 0.500000
interval score ration: 0.500000
data qulificaion report:


## 训练 Training

In [8]:
from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'

dataset_id = "./output_training_data/"
pretrain_work_dir = "./pretrain_work_dir/"

# 训练信息，用于指定需要训练哪个或哪些模型，这里展示AM和Vocoder模型皆进行训练
# 目前支持训练：TtsTrainType.TRAIN_TYPE_SAMBERT, TtsTrainType.TRAIN_TYPE_VOC
# 训练SAMBERT会以模型最新step作为基础进行finetune
train_info = {
    TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
        'train_steps': 202,               # 训练多少个step
        'save_interval_steps': 200,       # 每训练多少个step保存一次checkpoint
        'log_interval': 10               # 每训练多少个step打印一次训练日志
    }
}

# 配置训练参数，指定数据集，临时工作目录和train_info
kwargs = dict(
    model=pretrained_model_id,                  # 指定要finetune的模型
    model_revision = "v1.0.6",
    work_dir=pretrain_work_dir,                 # 指定临时工作目录
    train_dataset=dataset_id,                   # 指定数据集id
    train_type=train_info                       # 指定要训练类型及参数
)

trainer = build_trainer(Trainers.speech_kantts_trainer,
                        default_args=kwargs)

trainer.train()


2023-08-27 14:18:00,534 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 4.07k/4.07k [00:00<00:00, 1.28MB/s]
2023-08-27 14:18:03,476 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 138k/138k [00:00<00:00, 180kB/s]
Downloading: 100%|██████████| 288k/288k [00:00<00:00, 335kB/s]
Downloading: 100%|██████████| 327k/327k [00:00<00:00, 428kB/s]
Downloading: 100%|██████████| 535k/535k [00:01<00:00, 490kB/s]
Downloading: 100%|██████████| 506/506 [00:00<00:00, 2.59MB/s]
Downloading: 100%|██████████| 1.69M/1.69M [00:01<00:00, 1.25MB/s]
Downloading: 100%|██████████| 452k/452k [00:01<00:00, 290kB/s]
Downloading: 100%|██████████| 998k/998k [00:03<00:00, 260kB/s]
Downloading: 100%|██████████| 866M/866M [01:03<00:00, 14.3MB/s]
Downloading: 100%|██████████| 150M/150M [00:20<00:00, 7.76MB/s]
Downloading: 100%|██████████| 3.82k/3.82k [00:00<00:00, 2.06MB/s]
Downloading: 100%|██████████| 2.49k/2.49k [00:00<00:00, 7.

TextScriptConvertor.process:
Save script to: ./pretrain_work_dir/data/Script.xml
TextScriptConvertor.process:
Save metafile to: ./pretrain_work_dir/data/raw_metafile.txt
[AudioProcessor] Initialize AudioProcessor.
[AudioProcessor] config params:
[AudioProcessor] wav_normalize: True
[AudioProcessor] trim_silence: True
[AudioProcessor] trim_silence_threshold_db: 60
[AudioProcessor] preemphasize: False
[AudioProcessor] sampling_rate: 16000
[AudioProcessor] hop_length: 200
[AudioProcessor] win_length: 1000
[AudioProcessor] n_fft: 2048
[AudioProcessor] n_mels: 80
[AudioProcessor] fmin: 0.0
[AudioProcessor] fmax: 8000.0
[AudioProcessor] phone_level_feature: True
[AudioProcessor] se_feature: True
[AudioProcessor] norm_type: mean_std
[AudioProcessor] max_norm: 1.0
[AudioProcessor] symmetric: False
[AudioProcessor] min_level_db: -100.0
[AudioProcessor] ref_level_db: 20
[AudioProcessor] num_workers: 16
[AudioProcessor] Amplitude normalization started
Volume statistic proceeding...



100%|██████████| 19/19 [00:00<00:00, 55.33it/s]

Average amplitude RMS : 0.1696801578947368
Volume statistic done.
Volume normalization proceeding...



100%|██████████| 19/19 [00:00<00:00, 1570.37it/s]

...

100%|██████████| 1/1 [00:00<00:00,  1.15it/s]

Epoch 200 finished



  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 201 finished



2023-08-27 14:23:42,971 - modelscope - INFO - SAMBERT training spent: 0.03 hours

2023-08-27 14:23:42,973 - modelscope - INFO - skip HIFIGAN training...


## 推理 Inference （可编辑中文文本内容）

In [9]:
import os
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_dir = os.path.abspath("./pretrain_work_dir")

custom_infer_abs = {
    'voice_name':
    'F7',
    'am_ckpt':
    os.path.join(model_dir, 'tmp_am', 'ckpt'),
    'am_config':
    os.path.join(model_dir, 'tmp_am', 'config.yaml'),
    'voc_ckpt':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
    'voc_config':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
             'config.yaml'),
    'audio_config':
    os.path.join(model_dir, 'data', 'audio_config.yaml'),
    'se_file':
    os.path.join(model_dir, 'data', 'se', 'se.npy')
}
kwargs = {'custom_ckpt': custom_infer_abs}

model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

inference = pipeline(task=Tasks.text_to_speech, model=model_id)
output = inference(input="大家好呀，欢迎使用滔滔智能的声音克隆产品！") # 在这里编辑想要输出的文本

import IPython.display as ipd
ipd.Audio(output["output_wav"], rate=44100)


2023-08-27 14:23:42,986 - modelscope - INFO - am_config=/content/pretrain_work_dir/tmp_am/config.yaml voc_config=/content/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-08-27 14:23:42,987 - modelscope - INFO - audio_config=/content/pretrain_work_dir/data/audio_config.yaml
2023-08-27 14:23:42,988 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/content/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400200, '/content/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400200.pth')])
2023-08-27 14:23:42,990 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/content/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-08-27 14:23:42,991 - modelscope - INFO - se_path=/content/pretrain_work_dir/data/se/se.npy se_model_path=/content/pretrain_work_dir/orig_model/se.onnx
2023-08-27 14:23:42,992 - modelscope - INFO - mvn_path=/content/pretrain_work_dir/orig_model/mvn.npy
2023-08-27 14:23:48,733 - modelscope - WARNING - No prep

Removing weight norm...


## 打包训练好的模型，可下载到本地

In [ ]:
!zip -r /content/model.zip ./pretrain_work_dir

  adding: pretrain_work_dir/ (stored 0%)
  adding: pretrain_work_dir/tmp_voc/ (stored 0%)
  adding: pretrain_work_dir/tmp_am/ (stored 0%)
  adding: pretrain_work_dir/tmp_am/config.yaml (deflated 61%)
  adding: pretrain_work_dir/tmp_am/ckpt/ (stored 0%)
  adding: pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth (deflated 10%)
  adding: pretrain_work_dir/tmp_am/ckpt/checkpoint_2401000.pth (deflated 10%)
  adding: pretrain_work_dir/tmp_am/log/ (stored 0%)
  adding: pretrain_work_dir/tmp_am/log/events.out.tfevents.1688843636.977810d1baf5 (deflated 74%)
  adding: pretrain_work_dir/data/ (stored 0%)
  adding: pretrain_work_dir/data/frame_uv/ (stored 0%)
  adding: pretrain_work_dir/data/frame_uv/test_0_15.npy (deflated 83%)
  adding: pretrain_work_dir/data/frame_uv/test_0_0.npy (deflated 83%)
  adding: pretrain_work_dir/data/frame_uv/test_0_21.npy (deflated 91%)
  adding: pretrain_work_dir/data/frame_uv/test_0_10.npy (deflated 90%)
  adding: pretrain_work_dir/data/frame_uv/test_0_28.npy (

In [ ]:
from google.colab import files
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 也可以连接谷歌云盘，直接上传至云盘，速度更快

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

In [ ]:
shutil.copy("model.zip","/content/drive/MyDrive/folder_name")

'/content/drive/MyDrive/dataset/model.zip'